This notebook was original ran in a docker container where the project directory (i.e. same directory as README.md) is located in `/code`, which is set below. If you run locally you'll need to set the path of your project directory accordingly.

In [31]:
%cd /code

/code


---

The `load_dotenv()` function below loads all the variables found in the `.env` file as environment variables. You must have a `.env` file located in the project directory containing your OpenAI API key, in the following format.

```
OPENAI_API_KEY=sk-...
```

In [3]:
from dotenv import load_dotenv
load_dotenv()

def wprint(string: str, max_width: int = 80) -> None:
    import textwrap
    """Print `string` with a maximum widgth."""
    wrapped_string = textwrap.fill(string, max_width)
    print(wrapped_string)

# OpenAI Chat

Here's a simple example using `GPT-3.5` chat:

In [21]:
from llm_workflow.openai import OpenAIChat

model = OpenAIChat(model_name='gpt-3.5-turbo', temperature=0.2)
response_a = model("Write a function that returns the sum of two numbers.")
wprint(response_a)

Here is an example of a function that returns the sum of two numbers:  ```python
def add_numbers(num1, num2):     return num1 + num2 ```  You can use this
function by calling it and passing in two numbers as arguments. For example:
```python result = add_numbers(5, 3) print(result)  # Output: 8 ```


In [20]:
model = OpenAIChat(model_name='gpt-3.5-turbo', temperature=0.2)
response_b = model("Write a function that returns the sum of two numbers.")

wprint(response_b)

Sure! Here's an example of a function that takes two numbers as input and
returns their sum:  ```python def sum_numbers(num1, num2):     return num1 +
num2 ```  You can call this function by passing in two numbers as arguments,
like this:  ```python result = sum_numbers(5, 3) print(result)  # Output: 8 ```
In this example, the `sum_numbers` function takes two parameters `num1` and
`num2`, and returns their sum by using the `+` operator.


In [64]:
model.cost

0.00023799999999999998

In [99]:
from typing import Callable
import re

def extract_code_blocks(markdown_text: str) -> list:
    """Extract code blocks from Markdown text"""
    pattern = re.compile(r'```(?:python)?\s*(.*?)```', re.DOTALL)
    matches = pattern.findall(markdown_text)
    return [match.strip() for match in matches]


# def execute_code_blocks(code_blocks: list[str]) -> list[bool]:
#     """Execute code blocks and determine if the code blocks run successfully."""
#     block_results = []
#     local_namespace = {}
#     for code in code_blocks, start=1):
#         try:
#             _ = exec(code, local_namespace)
#             block_results.append(True)
#         except Exception as e:
#             block_results.append(False)
#     return block_results


import io
import contextlib

def execute_code_blocks(code_blocks: list[str]) -> list[bool]:
    """Execute code blocks and determine if the code blocks run successfully."""
    block_results = []
    local_namespace = {}
    for code in code_blocks:
        try:
            with contextlib.redirect_stdout(io.StringIO()):
                _ = exec(code, {}, local_namespace)
            block_results.append(True)
        except Exception as e:
            block_results.append(False)
    return block_results


class Trial:
    """TODO."""

    def __init__(
        self,
        chat_model: Callable[[str], str],
        prompts: list[str],
        description: str | None = None,
        ):  # noqa
        """TODO."""
        self._chat_model = chat_model
        self.prompts = prompts
        self.description = description
        self.responses = []
        self.duration_seconds = None
        
    
    def __call__(self) -> None:
        """TODO."""
        import time
        start = time.time()
        for prompt in self.prompts:
            self.responses.append(self._chat_model(prompt))
        end = time.time()
        self.duration_seconds = end - start
        self._code_block_results = [
            execute_code_blocks(code_blocks) for code_blocks in self.code_blocks
        ]
    
    def __str__(self) -> str:
        """TODO."""
        results = ""
        if self.description:
            results += f"{self.description}\n"
        results += f"Time: {self.duration_seconds:.2f} seconds\n"
        results += f"Characters: {self.num_characters:,}\n"
        results += f"Characters per second: {self.characters_per_second:.1f}\n"
        results += f"Num code blocks: {self.num_code_blocks}\n"
        percent_successful_code_blocks = (
            self.num_successful_code_blocks / self.num_code_blocks
        )
        results += f"Percent Passing Code blocks: {percent_successful_code_blocks:.1%}\n"
        if self.cost:
            results += f"Cost: ${self.cost:.5f}\n"
        return results
    
    @property
    def num_characters(self) -> int:
        """TODO."""
        return sum(len(response) for response in self.responses)

    @property
    def characters_per_second(self) -> float:
        """TODO."""
        return self.num_characters / self.duration_seconds

    @property
    def code_blocks(self) -> list[str]:
        """TODO."""
        return [extract_code_blocks(response) for response in self.responses]

    @property
    def num_code_blocks(self) -> int:
        """TODO."""
        return sum(len(code_blocks) for code_blocks in self.code_blocks)

    @property
    def num_successful_code_blocks(self) -> int:
        """TODO."""
        return sum(
            sum(code_block_results) for code_block_results in self._code_block_results
        )
    
    @property
    def cost(self) -> float:
        """TODO."""
        # if chat_model has a cost attribute, use that
        if hasattr(self._chat_model, 'cost'):
            return self._chat_model.cost
        return None
    

In [88]:
trial = Trial(
    OpenAIChat(model_name='gpt-3.5-turbo', temperature=0.0),
    prompts=['Write a function that returns the sum of two numbers.'],
    description='GPT-3.5 Turbo - Temperature 0.0',
)

In [89]:
trial()
print(trial)

Description: `GPT-3.5 Turbo`
Time: 3.05 seconds
Characters: 470
Characters per second: 154.18
Num code blocks: 2
Percent Passing Code blocks: 100.0%
Cost: $0.0003



In [90]:
display(Markdown(trial.responses[0]))

Sure! Here's an example of a function that takes two numbers as input and returns their sum:

```python
def add_numbers(num1, num2):
    return num1 + num2
```

You can use this function by calling it and passing in the two numbers you want to add together. For example:

```python
result = add_numbers(5, 3)
print(result)  # Output: 8
```

In this example, the function `add_numbers` takes two parameters `num1` and `num2`, and returns their sum using the `+` operator.

In [86]:
trial.code_blocks[0]

['def add_numbers(num1, num2):\n    return num1 + num2',
 'result = add_numbers(5, 3)\nprint(result)  # Output: 8']

In [91]:
class CompareTrials:
    def __init__(self, trials: list[list[Trial]]):
        self.trials = trials
    
    def __call__(self) -> str:
        for comparison in self.trials:
            for trial in comparison:
                trial()

In [93]:
prompt_1 = 'Write a function that returns the sum of two numbers.'
trials = [
    [
        Trial(
            OpenAIChat(model_name='gpt-3.5-turbo', temperature=0.0),
            prompts=[prompt_1],
            description='GPT-3.5 Turbo - Temperature 0.0',
        ),
        Trial(
            OpenAIChat(model_name='gpt-3.5-turbo', temperature=0.9),
            prompts=[prompt_1],
            description='GPT-3.5 Turbo - Temperature 0.9',
        )
    ],
]
comparison = CompareTrials(trials=trials)
comparison()

In [97]:
print(comparison.trials[0][0])

Description: `GPT-3.5 Turbo - Temperature 0.0`
Time: 2.25 seconds
Characters: 425
Characters per second: 188.59
Num code blocks: 2
Percent Passing Code blocks: 100.0%
Cost: $0.0003



In [98]:
print(comparison.trials[0][1])

Description: `GPT-3.5 Turbo - Temperature 0.9`
Time: 1.71 seconds
Characters: 430
Characters per second: 251.55
Num code blocks: 2
Percent Passing Code blocks: 100.0%
Cost: $0.0003



In [138]:
display(Markdown(comparison.trials[0][0].responses[0]))

Sure! Here's a simple function in Python that takes two numbers as input and returns their sum:

```python
def sum_numbers(num1, num2):
    return num1 + num2
```

You can use this function by calling it and passing in two numbers as arguments. For example:

```python
result = sum_numbers(5, 3)
print(result)  # Output: 8
```

Feel free to modify the function to suit your needs. Let me know if you have any other questions!

In [139]:
display(Markdown(comparison.trials[0][1].responses[0]))

Sure! Here's an example of a function that takes two numbers as parameters and returns their sum:

```
def sum_numbers(num1, num2):
    return num1 + num2
```

You can use this function by calling it and passing two numbers as arguments:

```
result = sum_numbers(3, 4)
print(result)  # Output: 7
```

In this example, the `sum_numbers` function takes two parameters `num1` and `num2` and returns their sum using the `+` operator.

In [129]:
def trial_to_html(trial: Trial) -> str:
    results = "<ul>\n"
    results += f"<li>Time: <code>{trial.duration_seconds:.2f}</code> seconds</li>\n"
    results += f"<li>Characters: <code>{trial.num_characters:,}</code></li>\n"
    results += f"<li>Characters per second: <code>{trial.characters_per_second:.1f}</code></li>\n"
    results += f"<li>Num code blocks: <code>{trial.num_code_blocks}</code></li>\n"
    percent_successful_code_blocks = (
        trial.num_successful_code_blocks / trial.num_code_blocks
    )
    results += f"<li>Percent Passing Code blocks: <code>{percent_successful_code_blocks:.1%}</code></li>\n"
    if trial.cost:
        results += f"<li>Cost: <code>${trial.cost:.5f}</code></li>\n"
    results += "</ul>\n"
    return results

trial_to_html(trial=comparison.trials[0][0])

'<ul>\n<li>Time: <code>2.25</code> seconds</li>\n<li>Characters: <code>425</code></li>\n<li>Characters per second: <code>188.6</code></li>\n<li>Num code blocks: <code>2</code></li>\n<li>Percent Passing Code blocks: <code>100.0%</code></li>\n<li>Cost: <code>$0.00026</code></li>\n</ul>\n'

In [167]:
import markdown
from pygments.formatters import HtmlFormatter
import textwrap

def markdown_to_html(trials: list[list[Trial]]):
    # Configure Markdown to HTML conversion
    md = markdown.Markdown(
        extensions=['fenced_code', 'codehilite'],
        extension_configs={
            'codehilite': {
                'css_class': 'highlight',
                'linenums': False,
                'use_pygments': True,
            }
        }
    )
    css = HtmlFormatter().get_style_defs('.highlight')

    # line_break = '<hr style="width:25%; text-align:left; margin-left:10px; height:1px; border-width:0>'  # noqa
    horizontal_line = '<div class="centered-line"></div>'
    # Generate rows and columns
    rows_html = ''
    column_names_html = ''
    for row in trials:
        columns_html = ''
        for trial in row:
            # columns_html += f'<td style="vertical-align: top;">{trial_to_html(trial)}</td>'
            html = md.convert(trial.responses[0])
            columns_html += f'<td style="vertical-align: top;">{trial_to_html(trial)}<br>{horizontal_line}<br>{html}</td>'
            if trial.description:
                column_names_html += f'<th>{trial.description}</th>'
                # <th style="width:50%;">Column 2</th>
            
        rows_html += f'<tr>{columns_html}</tr>'

    
    
    # Wrap the HTML and CSS in a complete HTML document with a table
    complete_html = textwrap.dedent(f'''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Document</title>
        <style>
        {css}
        .centered-line {{
            width: 50%;
            margin-left: 20px; /* Adjust this value to control the left alignment */
            border-top: 1px solid #000; /* You can adjust the color and style as needed */
        }}
        table {{
            width: 100%;
            border-collapse: collapse;
        }}
        th, td {{
            border: 1px solid #B2BEB5;
            padding: 8px;
            text-align: left;
        }}
        </style>
    </head>
    <body>
    <table border="1" style="width:100%; border-collapse: collapse;">
        <thead>
            <tr>
                {column_names_html}
            </tr>
        </thead>
        {rows_html}
    </table>
    </body>
    </html>
    ''')
    
    return complete_html


In [168]:
comparison_html = markdown_to_html(comparison.trials)
with open("/code/examples/comparison_example.html", "w") as file:
    file.write(comparison_html)
display(HTML(comparison_html))


GPT-3.5 Turbo - Temperature 0.0,GPT-3.5 Turbo - Temperature 0.9
"Time: 2.25 seconds Characters: 425 Characters per second: 188.6 Num code blocks: 2 Percent Passing Code blocks: 100.0% Cost: $0.00026 Sure! Here's a simple function in Python that takes two numbers as input and returns their sum: def sum_numbers(num1, num2): return num1 + num2 You can use this function by calling it and passing in two numbers as arguments. For example: result = sum_numbers(5, 3) print(result) # Output: 8 Feel free to modify the function to suit your needs. Let me know if you have any other questions!","Time: 1.71 seconds Characters: 430 Characters per second: 251.6 Num code blocks: 2 Percent Passing Code blocks: 100.0% Cost: $0.00027 Sure! Here's an example of a function that takes two numbers as parameters and returns their sum: def sum_numbers(num1, num2): return num1 + num2 You can use this function by calling it and passing two numbers as arguments: result = sum_numbers(3, 4) print(result) # Output: 7 In this example, the sum_numbers function takes two parameters num1 and num2 and returns their sum using the + operator."


In [162]:
print(comparison.trials[0][1].responses[0])

Sure! Here's an example of a function that takes two numbers as parameters and returns their sum:

```
def sum_numbers(num1, num2):
    return num1 + num2
```

You can use this function by calling it and passing two numbers as arguments:

```
result = sum_numbers(3, 4)
print(result)  # Output: 7
```

In this example, the `sum_numbers` function takes two parameters `num1` and `num2` and returns their sum using the `+` operator.


In [18]:
import difflib


def _create_html_difference_list(value_a: str, value_b: str) -> str:
    """
    Given value_a and value_b return unified_diff, after cleaning up what unified_diff returns.
    Args:
        value_a:
        value_b:
    """
    diff = list(difflib.unified_diff(a=value_a, b=value_b, n=1000))
    if len(diff) == 0:  # equal strings
        diff = [" " + char for char in value_a]
    else:
        diff = diff[3:]
    return diff


def _create_html_change_span(value: str, is_change: bool, change_color: str = '#F1948A') -> str:
    """
    Args:
        value: a single character
        is_change: if True, highlight the character in red
        change_color: color of background to highlight differences.
    Returns:
        e.g. "<span style="background:#ffe6e6";>value</span>"
    """
    background_color = ''
    if is_change:
        background_color = f' style="background:{change_color}";'
    return f'<span{background_color}>{value}</span>'


def _create_html_cell(difference_list: list, is_first_value: bool, change_color: str = '#F1948A'):
    """
    Creates a single cell (e.g. name, domain, etc.) from one company's differences.
    Args:
        difference_list: list returned from create_difference_list
        is_first_value: if True, treats difference_list according to first value.
        change_color: color of background to highlight differences.
    """
    if is_first_value:
        diff = [(x[1:], x[0] != ' ') for x in difference_list if x[0] in [' ', '-']]
    else:
        diff = [(x[1:], x[0] != ' ') for x in difference_list if x[0] in [' ', '+']]

    html = [
        _create_html_change_span(value=x[0], is_change=x[1], change_color=change_color)
        for x in diff
    ]
    return ''.join(html)


def diff_text(
        text_a: str | list[str],
        text_b: str | list[str],
        change_color: str = '#F1948A') -> str:
    """
    Returns string as HTML containing highlighted differences between `text_a` and
    `text_b`.

    The HTML will contain a table with a single column that that contains `text_a` on top and
    `text_b` on the bottom.

    All `new line` characters are removed and replaced with a space.

    Args:
        text_a: this text will be represented on the top of each html cell.
        text_b: this text will be represented on the bottom of each html
        cell. change_color: color of background to highlight differences.
    """
    html = '''
    <html>
    <head>
      <style>
        table, th, td { border: 1px solid black; border-collapse: collapse; white-space: normal;}
        .markdown-container {
            font-family: Arial, sans-serif;
            padding: 10px;
            border: 1px solid #ccc;
        }
      </style>
    </head>
    <body style="font-family: monospace">
    '''
    html += '<script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>'
    html += """
    <script>
    // Get references to all elements with the "markdown-container" class
    const markdownContainers = document.querySelectorAll(".markdown-container");

    // Your Markdown content
    const markdownText = `
    # Hello, Markdown!

    This is some **Markdown** text.

    - List item 1
    - List item 2
    `;

    // Convert and render the Markdown for each container
    markdownContainers.forEach((container) => {
        container.innerHTML = marked(markdownText);
    });
    </script>
    """
    html += '<table><tr>'
    html += '<th>index</th>'
    html += '<th>diff</th>'
    html += '</tr>'

    # line_break = '<hr style="width:25%; text-align:left; margin-left:10px; height:1px; ' \
    #     'border-width:0;' \
    #     'color:blue; background-color:blue">'
    line_break = '<hr style="border: none; border-left: 1px solid #000; height: 100px;">'

    def create_inline_change(diff_list):
        diff_a = _create_html_cell(
            difference_list=diff_list, is_first_value=True, change_color=change_color
        )
        diff_b = _create_html_cell(
            difference_list=diff_list, is_first_value=False, change_color=change_color
        )
        return f'<td class="markdown-container">{diff_a}</td>' + f'<td>{diff_b}</td>'

    if isinstance(text_a, str):
        assert isinstance(text_b, str)
        text_a = [text_a]
        text_b = [text_b]
    else:
        assert len(text_a) == len(text_b)

    for index in range(len(text_a)):
        html += '<tr>'
        html += f"<td>{index}</td>"
        difference_list = _create_html_difference_list(
            value_a=text_a[index],
            value_b=text_b[index]
        )
        html += create_inline_change(diff_list=difference_list)
        html += '</tr>'
    html += "</table></body></html>"
    return html


In [27]:
print(response_a)

Here is an example of a function that returns the sum of two numbers:

```python
def add_numbers(num1, num2):
    return num1 + num2
```

You can use this function by calling it and passing in two numbers as arguments. For example:

```python
result = add_numbers(5, 3)
print(result)  # Output: 8
```


In [23]:
from IPython.display import display, HTML, Markdown
display(Markdown(response_a))
display(Markdown(response_b))

Here is an example of a function that returns the sum of two numbers:

```python
def add_numbers(num1, num2):
    return num1 + num2
```

You can use this function by calling it and passing in two numbers as arguments. For example:

```python
result = add_numbers(5, 3)
print(result)  # Output: 8
```

Sure! Here's an example of a function that takes two numbers as input and returns their sum:

```python
def sum_numbers(num1, num2):
    return num1 + num2
```

You can call this function by passing in two numbers as arguments, like this:

```python
result = sum_numbers(5, 3)
print(result)  # Output: 8
```

In this example, the `sum_numbers` function takes two parameters `num1` and `num2`, and returns their sum by using the `+` operator.

In [25]:
# display html
display(HTML(diff_text(response_a, response_b)))

In [28]:
import markdown

ModuleNotFoundError: No module named 'markdown'